In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['wikisql', 'hindienglish-corpora']


In [3]:
lines=pd.read_csv("../input/wikisql/train.csv",encoding='utf-8')

In [4]:
lines.head(20)

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA
1,What is the current series where the new series began in June 2011?,SELECT Current series FROM table WHERE Notes = New series began in June 2011
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory = South Australia
3,Name the background colour for the Australian Capital Territory,SELECT Text/background colour FROM table WHERE State/territory = Australian Capital Territory
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM table WHERE Fuel Propulsion = CNG
5,what is the fuel propulsion where the fleet series (quantity) is 310-329 (20)?,SELECT Fuel Propulsion FROM table WHERE Fleet Series (Quantity) = 310-329 (20)
6,who is the manufacturer for the order year 1998?,SELECT Manufacturer FROM table WHERE Order Year = 1998
7,how many times is the model ge40lfr?,SELECT COUNT Manufacturer FROM table WHERE Model = GE40LFR
8,how many times is the fleet series (quantity) is 468-473 (6)?,SELECT COUNT Order Year FROM table WHERE Fleet Series (Quantity) = 468-473 (6)
9,what is the powertrain (engine/transmission) when the order year is 2000?,SELECT Powertrain (Engine/Transmission) FROM table WHERE Order Year = 2000


In [5]:
pd.isnull(lines).sum()

question    0
sql         0
dtype: int64

In [6]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 35000 rows from the dataset.

In [7]:
lines=lines.sample(n=35000,random_state=42)
lines.shape

(35000, 2)

In [8]:
# Lowercase all characters
lines['question']=lines['question'].apply(lambda x: x.lower())
lines['sql']=lines['sql'].apply(lambda x: x.lower())

In [9]:
# Remove extra spaces
lines['question']=lines['question'].apply(lambda x: x.strip())
lines['sql']=lines['sql'].apply(lambda x: x.strip())
lines['question']=lines['question'].apply(lambda x: re.sub(" +", " ", x))
lines['sql']=lines['sql'].apply(lambda x: re.sub(" +", " ", x))


In [10]:
# Add start and end tokens to target sequences
lines['sql'] = lines['sql'].apply(lambda x : 'START_ '+ x + ' _END')

In [11]:
lines.head()

,question,sql
43648,how much is the purse worth (¥) after 2012?,START_ select sum purse (¥) from table where year > 2012 _END
46173,what is the name for malay with a foundation in iptura?,START_ select name in malay from table where foundation = iptura _END
19352,what is the highest weight for the farnley stakes race?,START_ select max weight (kg) from table where race = farnley stakes _END
3130,what are the won games with losing bonus of 0?,START_ select won from table where losing bonus = 0 _END
3192,what's the airport with % change 2005/2006 being 13.0%,START_ select airport from table where % change 2005/2006 = 13.0% _END


In [ ]:
# Get Question and SQL Vocab
all_questions_words=set()
for question in lines['question']:
    for word in question.split():
        if word not in all_questions_words:
            all_questions_words.add(word)

all_sql_words=set()
for sql in lines['sql']:
    for word in sql.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

In [13]:
len(all_questions_words)

38969

In [14]:
len(all_sql_words)

30712

In [15]:
lines['length_question']=lines['question'].apply(lambda x:len(x.split(" "))) # Number of words in a question
lines['length_sql']=lines['sql'].apply(lambda x:len(x.split(" "))) # Number of words in a sql

In [16]:
lines.head()

,question,sql,length_question,length_sql
43648,how much is the purse worth (¥) after 2012?,START_ select sum purse (¥) from table where year > 2012 _END,9,12
46173,what is the name for malay with a foundation in iptura?,START_ select name in malay from table where foundation = iptura _END,11,12
19352,what is the highest weight for the farnley stakes race?,START_ select max weight (kg) from table where race = farnley stakes _END,10,13
3130,what are the won games with losing bonus of 0?,START_ select won from table where losing bonus = 0 _END,10,11
3192,what's the airport with % change 2005/2006 being 13.0%,START_ select airport from table where % change 2005/2006 = 13.0% _END,5,12


In [17]:
lines[lines['length_question']>30].shape

(75, 4)

In [18]:
lines=lines[lines['length_question']<=20]
lines=lines[lines['length_sql']<=20]

In [19]:
lines.shape

(32775, 4)

In [20]:
print("maximum length of SQL Sentence ",max(lines['length_sql']))
print("maximum length of Question Sentence ",max(lines['length_question']))

maximum length of SQL Sentence  20
maximum length of Question Sentence  20


In [21]:
max_length_src=max(lines['length_sql'])
max_length_tar=max(lines['length_question'])

In [22]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(38969, 30712)

In [23]:
num_decoder_tokens += 1 #for zero padding


In [24]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [25]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [26]:
lines = shuffle(lines)
lines.head(30)

,question,sql,length_question,length_sql
16750,what was the grid for bruno junqueira for dale coyne racing?,START_ select grid from table where team = dale coyne racing and driver = bruno junqueira _END,11,17
35684,what was the date of the game when atlanta was the home team?,START_ select date from table where home = atlanta _END,13,10
839,which source shows blackberry at 2.9%?,START_ select source from table where blackberry = 2.9% _END,6,10
53853,which country has a finish of t4?,START_ select country from table where finish = t4 _END,7,10
3599,what is the percentage of literate people where india is andaman and nicobar islands?,START_ select literate persons (%) from table where india/state/ut = andaman and nicobar islands _END,14,15
45101,what was the tie number when bournemouth was the home team?,START_ select tie no from table where home team = bournemouth _END,11,12
47743,what's the 2008 status of rosa delauro?,START_ select 2008 status from table where democratic = rosa delauro _END,7,12
44205,what is 2008 when the tournament is year end ranking?,START_ select 2008 from table where tournament = year end ranking _END,10,12
7539,"from writer denne bart petitclerc, what is the maximum season number?",START_ select max season # from table where writer(s) = denne bart petitclerc _END,11,14
46749,"what is date, when venue is ""taunton"", and when name is ""sc ganguly""?",START_ select date from table where venue = taunton and name = sc ganguly _END,13,15


### Split the data into train and test

In [27]:
X, y = lines['question'], lines['sql']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((26220,), (6555,))

### Let us save this data

In [28]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [29]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [30]:
latent_dim=300

In [31]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [32]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [33]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [34]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11690700    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    9213900     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [35]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 20

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
409/409 [==============================] - 158s 385ms/step - loss: 4.0075 - val_loss: 3.3643
Epoch 2/20
259/409 [=================>............] - ETA: 46s - loss: 3.1653

In [ ]:
import joblib
# save the model to disk
filename = 'model.sav'
joblib.dump(model, filename)

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.summary()

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 200):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
f_question_file = open("question.txt","w")
sql_file = open("sql.txt","w")
predection_file = open("predection.txt","w")

In [ ]:
df = pd.DataFrame(columns = ['Input qustion sentence', 'Actual sql Translation', 'Prideccted'])
for x in range(10000):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    #print('Input English sentence:', X_train[k:k+1].values[0])
    #print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
    #print('Predicted Hindi Translation:', decoded_sentence[:-4])
    data_dict = {
        "Input qustion sentence": X_train[k:k+1].values[0],
        "Actual sql Translation": y_train[k:k+1].values[0][6:-4],
        "Prideccted": decoded_sentence[:-4]
    }
    
    df = df.append(data_dict,ignore_index = True)
    
    
    f_question_file = open("question.txt","a")
    sql_file = open("sql.txt","a")
    predection_file = open("predection.txt","a")

    f_question_file.write(X_train[k:k+1].values[0]+'\n')
    sql_file.write(y_train[k:k+1].values[0][6:-4]+'\n')
    predection_file.write(decoded_sentence[:-4]+'\n')
    
    
    # =X_train[k:k+1].values[0]
    #rint(t)
    #ist_str=[]
    #ist_str.append(t)
    #rint(list_str) 
    #f = pd.DataFrame({"Input qustion sentence": list_str})
    #     print('-------------------------------------')


    f_question_file.close()
    sql_file.close()
    predection_file.close()
df.to_csv('resultMachine.csv')
